In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

from keras import layers
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, auc, roc_curve, roc_auc_score
from sklearn.preprocessing import StandardScaler
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
import pandas as pd
from keras.layers import LSTM, Dense
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


In [3]:
df = pd.read_csv('Preprocessing/fight_with_stats_precomp.csv')
df_fighters = pd.read_csv('Preprocessing/fighter_total_stats.csv')

In [4]:
df_fighters_details = pd.read_csv('Preprocessing/fighter_details.csv', parse_dates=True)

In [5]:
df = df[(df['fightNo_fighter']>=3) & (df['fightNo_opponent']>=3) & (df['Women_fight']==0) & (df['ROUND']<4)]

In [6]:
df = df.fillna(0)


In [7]:
df.head()

,EVENT,BOUT,FIGHTER,OPPONENT,METHOD,Win,Women_fight,ROUND,HEIGHT_diff,REACH_diff,...,BODY_landed_opponent,BODY_attemps_opponent,LEG_landed_opponent,LEG_attemps_opponent,DISTANCE_landed_opponent,DISTANCE_attemps_opponent,CLINCH_landed_opponent,CLINCH_attemps_opponent,GROUND_landed_opponent,GROUND_attemps_opponent
24,UFC 2: No Way Out,Royce Gracie vs. Patrick Smith,Patrick Smith,Royce Gracie,KO/TKO - ground,0,0,1,1.0,0.0,...,0.0,0.0,3.0,5.0,5.0,10.0,0.0,0.0,0.0,0.0
25,UFC 2: No Way Out,Royce Gracie vs. Patrick Smith,Royce Gracie,Patrick Smith,KO/TKO - ground,1,0,1,-1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,3.0,5.0
26,UFC 2: No Way Out,Royce Gracie vs. Remco Pardoel,Remco Pardoel,Royce Gracie,Submission,0,0,1,2.0,0.0,...,0.0,0.0,1.0,1.0,2.0,3.0,0.0,0.0,0.0,2.0
27,UFC 2: No Way Out,Royce Gracie vs. Remco Pardoel,Royce Gracie,Remco Pardoel,Submission,1,0,1,-2.0,0.0,...,1.0,2.0,0.0,1.0,0.0,3.0,1.0,1.0,3.0,5.0
50,UFC 4: Revenge of the Warriors,Royce Gracie vs. Dan Severn,Dan Severn,Royce Gracie,Submission,0,0,1,1.0,0.0,...,3.0,4.0,2.0,3.0,3.0,5.0,3.0,3.0,3.0,3.0


In [8]:
df.shape

(6361, 63)

In [9]:
df.columns.tolist()[19:]

['KD_fighter',
 'SUB.ATT_fighter',
 'REV._fighter',
 'CTRL_fighter',
 'SIG.STR._landed_fighter',
 'SIG.STR._attemps_fighter',
 'TOTAL STR._landed_fighter',
 'TOTAL STR._attemps_fighter',
 'TD_landed_fighter',
 'TD_attemps_fighter',
 'HEAD_landed_fighter',
 'HEAD_attemps_fighter',
 'BODY_landed_fighter',
 'BODY_attemps_fighter',
 'LEG_landed_fighter',
 'LEG_attemps_fighter',
 'DISTANCE_landed_fighter',
 'DISTANCE_attemps_fighter',
 'CLINCH_landed_fighter',
 'CLINCH_attemps_fighter',
 'GROUND_landed_fighter',
 'GROUND_attemps_fighter',
 'KD_opponent',
 'SUB.ATT_opponent',
 'REV._opponent',
 'CTRL_opponent',
 'SIG.STR._landed_opponent',
 'SIG.STR._attemps_opponent',
 'TOTAL STR._landed_opponent',
 'TOTAL STR._attemps_opponent',
 'TD_landed_opponent',
 'TD_attemps_opponent',
 'HEAD_landed_opponent',
 'HEAD_attemps_opponent',
 'BODY_landed_opponent',
 'BODY_attemps_opponent',
 'LEG_landed_opponent',
 'LEG_attemps_opponent',
 'DISTANCE_landed_opponent',
 'DISTANCE_attemps_opponent',
 'CLINCH

In [10]:
df = df.merge(df[['EVENT', 'BOUT', 'FIGHTER']+df.columns.tolist()[15:]] ,how='left', left_on=['EVENT', 'BOUT', 'OPPONENT'], right_on=['EVENT', 'BOUT', 'FIGHTER'], suffixes=('_f1', '_f2'))

In [11]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6361 entries, 0 to 6360
Data columns (total 112 columns):
 #    Column                          Non-Null Count  Dtype  
---   ------                          --------------  -----  
 0    EVENT                           6361 non-null   object 
 1    BOUT                            6361 non-null   object 
 2    FIGHTER_f1                      6361 non-null   object 
 3    OPPONENT                        6361 non-null   object 
 4    METHOD                          6361 non-null   object 
 5    Win                             6361 non-null   int64  
 6    Women_fight                     6361 non-null   int64  
 7    ROUND                           6361 non-null   int64  
 8    HEIGHT_diff                     6361 non-null   float64
 9    REACH_diff                      6361 non-null   float64
 10   AGE_diff                        6361 non-null   float64
 11   form_skore_fighter              6361 non-null   float64
 12   form_skore_opponen

In [12]:
df = df.drop('FIGHTER_f2',axis=1)

In [13]:
'LEG_attemps_fighter_f1'.split('_')

['LEG', 'attemps', 'fighter', 'f1']

In [14]:
sloupce1 = df.columns.tolist()[19:]
vynechat = ['Win_tot_f2', 'DRAW_f2','No_contest_f2']
sloupce = [x for x in sloupce1 if x not in vynechat]
for sloupec in sloupce:    
    splited = sloupec.split('_')
    if 'CTRL' in splited:
        df.loc[:,splited[0]+'_pct_'+splited[1]+splited[2]] = df[sloupec]/df['TotalTime'+'_'+splited[2]]
    if 'attemps' in splited:
        df.loc[:,splited[0]+'_acc_'+ splited[2] + splited[3]] = df[sloupec.replace('attemps', 'landed')]/df[sloupec]
        df.loc[:,splited[0]+'_perRound_'+ splited[2] + splited[3]] = (df[sloupec.replace('attemps', 'landed')]/df['TotalTime'+'_'+splited[3]])*300
        
    

In [15]:
df.iloc[:,:5]

,EVENT,BOUT,FIGHTER_f1,OPPONENT,METHOD
0,UFC 2: No Way Out,Royce Gracie vs. Patrick Smith,Patrick Smith,Royce Gracie,KO/TKO - ground
1,UFC 2: No Way Out,Royce Gracie vs. Patrick Smith,Royce Gracie,Patrick Smith,KO/TKO - ground
2,UFC 2: No Way Out,Royce Gracie vs. Remco Pardoel,Remco Pardoel,Royce Gracie,Submission
3,UFC 2: No Way Out,Royce Gracie vs. Remco Pardoel,Royce Gracie,Remco Pardoel,Submission
4,UFC 4: Revenge of the Warriors,Royce Gracie vs. Dan Severn,Dan Severn,Royce Gracie,Submission
...,...,...,...,...,...
6356,UFC Fight Night: Lewis vs. Spivac,Derrick Lewis vs. Serghei Spivac,Serghei Spivac,Derrick Lewis,Submission
6357,UFC Fight Night: Lewis vs. Spivac,Dooho Choi vs. Kyle Nelson,Dooho Choi,Kyle Nelson,Decision
6358,UFC Fight Night: Lewis vs. Spivac,Dooho Choi vs. Kyle Nelson,Kyle Nelson,Dooho Choi,Decision
6359,UFC Fight Night: Lewis vs. Spivac,Marcin Tybura vs. Blagoy Ivanov,Blagoy Ivanov,Marcin Tybura,Decision


In [16]:
#df['Avg_time1'] = df['TotalTime_f1']/df['fightNo_fighter']
#df['Avg_time2'] = df['TotalTime_f2']/df['fightNo_opponent']

In [17]:
df = df.drop(sloupce, axis=1)

In [18]:
df_model = df.drop(['EVENT','BOUT','FIGHTER_f1','OPPONENT','METHOD','Women_fight','TotalTime_f1', 'Win'],axis=1)

In [19]:
df_model = df_model.fillna(0)
df_model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6361 entries, 0 to 6360
Data columns (total 90 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ROUND                           6361 non-null   int64  
 1   HEIGHT_diff                     6361 non-null   float64
 2   REACH_diff                      6361 non-null   float64
 3   AGE_diff                        6361 non-null   float64
 4   form_skore_fighter              6361 non-null   float64
 5   form_skore_opponent             6361 non-null   float64
 6   fightNo_fighter                 6361 non-null   int64  
 7   fightNo_opponent                6361 non-null   float64
 8   Win_tot_f1                      6361 non-null   float64
 9   DRAW_f1                         6361 non-null   float64
 10  No_contest_f1                   6361 non-null   float64
 11  Win_tot_f2                      6361 non-null   float64
 12  DRAW_f2                         63

In [20]:
X=df_model.loc[:, ~df_model.columns.isin(['ROUND'])]
y=df_model['ROUND']
y= np.array(y)-1
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state = 0)
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression(multi_class='auto')
logreg.fit(X_train,y_train)
predictions = logreg.predict(X_test)
acc = accuracy_score(y_test, predictions) 
f1 = f1_score(y_test, predictions,average='micro')
print(f1, acc)

0.5468258956631049 0.5468258956631049


In [21]:
a = np.array(X.iloc[-20,:])
a.reshape(1,-1).shape


(1, 89)

In [22]:
logreg.predict_proba(a.reshape(1,-1))

array([[2.83323795e-01, 1.43162110e-04, 7.16533043e-01]])

In [25]:
np.unique(predictions)

array([0, 1, 2], dtype=int64)

In [23]:
X.isna().sum().sort_values()

HEIGHT_diff                   0
LEG_perRound_fighterf2        0
LEG_acc_fighterf2             0
BODY_perRound_fighterf2       0
BODY_acc_fighterf2            0
                             ..
LEG_perRound_fighterf1        0
LEG_acc_fighterf1             0
BODY_perRound_fighterf1       0
LEG_acc_opponentf1            0
GROUND_perRound_opponentf2    0
Length: 89, dtype: int64

In [115]:
X_train.shape

(4754, 89)

In [116]:
train_y_one_hot = keras.utils.to_categorical(y_train)
test_y_one_hot = keras.utils.to_categorical(y_test)

In [117]:
test_y_one_hot.shape

(1585, 3)

In [124]:
class_neur = MLPClassifier(random_state=1, max_iter=100)
model = Sequential()
model.add(layers.Dense(10, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.4))
model.add(layers.Dense(10, activation='relu'))
model.add(Dropout(0.2))
model.add(layers.Dense(3, activation='softmax'))
model.compile(optimizer = 'adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])  
history =model.fit(X_train, train_y_one_hot, epochs=50, batch_size = 8, verbose=1,validation_data=(X_test, test_y_one_hot))


Epoch 1/50
595/595 [==============================] - 2s 2ms/step - loss: 0.6342 - accuracy: 0.5011 - val_loss: 0.5997 - val_accuracy: 0.5249
Epoch 2/50
595/595 [==============================] - 1s 1ms/step - loss: 0.6022 - accuracy: 0.5322 - val_loss: 0.5947 - val_accuracy: 0.5249
Epoch 3/50
595/595 [==============================] - 1s 1ms/step - loss: 0.5933 - accuracy: 0.5358 - val_loss: 0.5913 - val_accuracy: 0.5287
Epoch 4/50
595/595 [==============================] - 1s 1ms/step - loss: 0.5879 - accuracy: 0.5370 - val_loss: 0.5889 - val_accuracy: 0.5300
Epoch 5/50
595/595 [==============================] - 1s 1ms/step - loss: 0.5836 - accuracy: 0.5408 - val_loss: 0.5876 - val_accuracy: 0.5331
Epoch 6/50
595/595 [==============================] - 1s 1ms/step - loss: 0.5847 - accuracy: 0.5379 - val_loss: 0.5863 - val_accuracy: 0.5350
Epoch 7/50
595/595 [==============================] - 1s 1ms/step - loss: 0.5811 - accuracy: 0.5448 - val_loss: 0.5847 - val_accuracy: 0.5344
Epoch 

In [202]:
y_pred=model.predict(X_test)

50/50 [==============================] - 0s 714us/step


In [215]:
y_test

array([2, 2, 0, ..., 0, 2, 1], dtype=int64)

In [213]:
list_pred = np.array([np.argmax(proba) for proba in y_pred])
list_pred

array([2, 2, 2, ..., 0, 2, 2], dtype=int64)

In [219]:
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix
print(classification_report(y_test, list_pred))

              precision    recall  f1-score   support

           0       0.48      0.16      0.24       448
           1       0.00      0.00      0.00       301
           2       0.55      0.94      0.69       836

    accuracy                           0.54      1585
   macro avg       0.34      0.37      0.31      1585
weighted avg       0.42      0.54      0.43      1585



D:\anacinda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anacinda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anacinda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [203]:
from sklearn.metrics import confusion_matrix

In [177]:
X_test.shape

(1585, 89)

In [180]:
np.mean(y_pred_proba)

0.33333334

In [181]:
model.save('model_round.h5')
model.save_weights('model_round.h5')  # to store


In [200]:
y_pred = model.predict(a.reshape(1,-1))

1/1 [==============================] - 0s 23ms/step


In [221]:
y_pred

array([[0.2560396 , 0.17478523, 0.5691751 ],
       [0.18876061, 0.14398833, 0.66725105],
       [0.17153977, 0.11897703, 0.70948315],
       ...,
       [0.39213213, 0.21862605, 0.38924184],
       [0.16401476, 0.0974431 , 0.7385422 ],
       [0.33668017, 0.21557921, 0.44774064]], dtype=float32)

In [222]:
from datetime import date

today = date.today()
df_fighters_details['AGE'] = (pd.to_datetime(today) - pd.to_datetime(df_fighters_details['DOB'])).astype('<m8[Y]')

In [255]:
fighter2 = "Da-Un Jung"
fighter1= "Devin Clark"


In [256]:
f1_df = df_fighters.loc[df_fighters['FIGHTER']==fighter1]
f2_df = df_fighters.loc[df_fighters['FIGHTER']==fighter2]

In [257]:
f1_df

,FIGHTER,HEIGHT_fighter,WEIGHT_fighter,REACH_fighter,STANCE_fighter,Fights,Win,DRAW,Lost,No_contest,...,BODY_attemps_opponent,LEG_landed_opponent,LEG_attemps_opponent,DISTANCE_landed_opponent,DISTANCE_attemps_opponent,CLINCH_landed_opponent,CLINCH_attemps_opponent,GROUND_landed_opponent,GROUND_attemps_opponent,form_skore_fighter
600,Devin Clark,72.0,205.0,75.0,Orthodox,14,7,0,7,0,...,141.0,54.0,62.0,175.0,447.0,155.0,198.0,117.0,155.0,-0.5


In [258]:
df_model.columns

Index(['ROUND', 'HEIGHT_diff', 'REACH_diff', 'AGE_diff', 'form_skore_fighter',
       'form_skore_opponent', 'fightNo_fighter', 'fightNo_opponent',
       'Win_tot_f1', 'DRAW_f1', 'No_contest_f1', 'Win_tot_f2', 'DRAW_f2',
       'No_contest_f2', 'CTRL_pct_fighterf1', 'SIG.STR._acc_fighterf1',
       'SIG.STR._perRound_fighterf1', 'TOTAL STR._acc_fighterf1',
       'TOTAL STR._perRound_fighterf1', 'TD_acc_fighterf1',
       'TD_perRound_fighterf1', 'HEAD_acc_fighterf1',
       'HEAD_perRound_fighterf1', 'BODY_acc_fighterf1',
       'BODY_perRound_fighterf1', 'LEG_acc_fighterf1',
       'LEG_perRound_fighterf1', 'DISTANCE_acc_fighterf1',
       'DISTANCE_perRound_fighterf1', 'CLINCH_acc_fighterf1',
       'CLINCH_perRound_fighterf1', 'GROUND_acc_fighterf1',
       'GROUND_perRound_fighterf1', 'CTRL_pct_opponentf1',
       'SIG.STR._acc_opponentf1', 'SIG.STR._perRound_opponentf1',
       'TOTAL STR._acc_opponentf1', 'TOTAL STR._perRound_opponentf1',
       'TD_acc_opponentf1', 'TD_perRoun

In [259]:
agediff = df_fighters_details[df_fighters_details['FIGHTER']==fighter1]['AGE'].values[0] - df_fighters_details[df_fighters_details['FIGHTER']==fighter2]['AGE'].values[0]
formy = [f1_df['form_skore_fighter'].values[0], f2_df['form_skore_fighter'].values[0]]
no_of_fights = [f1_df['Fights'].values[0],f2_df['Fights'].values[0]]
W_D_NC = f1_df[['Win','DRAW', 'No_contest']].values.tolist()[0] + f2_df[['Win','DRAW', 'No_contest']].values.tolist()[0]

In [260]:
sloupce2 = df_fighters.columns.tolist()[10:]
stats_f1 = []
stats_f2 = []
for sloupec in sloupce2:    
    splited = sloupec.split('_')
    if 'CTRL' in splited:
        stats_f1.append((f1_df[sloupec]/f1_df['TotalTime']).values[0])
        stats_f2.append((f2_df[sloupec]/f2_df['TotalTime']).values[0])
    if 'attemps' in splited:
        stats_f1.append((f1_df[sloupec.replace('attemps', 'landed')]/f1_df[sloupec]).values[0])
        stats_f1.append((f1_df[sloupec.replace('attemps', 'landed')]/f1_df['TotalTime']).values[0]*300)
        
        stats_f2.append((f2_df[sloupec.replace('attemps', 'landed')]/f2_df[sloupec]).values[0])
        stats_f2.append((f2_df[sloupec.replace('attemps', 'landed')]/f2_df['TotalTime']).values[0]*300)
stats_list = stats_f1 + stats_f2

In [261]:
vstup = np.array([1]+[f1_df.iloc[0][col] - f2_df.iloc[0][col] for col in ['HEIGHT_fighter','REACH_fighter']] + [agediff] + formy + no_of_fights + W_D_NC + stats_list)

In [262]:
scaler = MinMaxScaler(feature_range=(0, 1))
vstup_scaled = scaler.fit_transform(df_model.append(pd.DataFrame(vstup.reshape(1,-1), columns=list(df_model)), ignore_index=True))[-1:,1:]

C:\Users\spravce\AppData\Local\Temp\ipykernel_175276\1100220427.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  vstup_scaled = scaler.fit_transform(df_model.append(pd.DataFrame(vstup.reshape(1,-1), columns=list(df_model)), ignore_index=True))[-1:,1:]


In [263]:
vstup_scaled.shape

(1, 89)

In [264]:
from numpy import *
where_are_NaNs = isnan(vstup_scaled)
vstup_scaled[where_are_NaNs] = 0

In [265]:
np.argwhere(np.isnan(vstup_scaled))

array([], shape=(0, 2), dtype=int64)

In [266]:
y_pred = model.predict(vstup_scaled)
y_pred

1/1 [==============================] - 0s 20ms/step


array([[0.21610133, 0.14898303, 0.6349156 ]], dtype=float32)